In [27]:

import numpy as np
from backend.data_center.kline_data.kline_data_collector import *

tv = KlineDataCollector()
file_abspath = tv.get_abspath(symbol='BTC', interval=Interval.in_daily)
df = pd.read_csv(f"{file_abspath}")


error while signin
you are using nologin method, data you access may be limited


In [28]:
def perfect_order_strategy(df: DataFrame) -> DataFrame:
    if 'high' not in df.columns or 'low' not in df.columns or 'close' not in df.columns:
        raise ValueError("DataFrame must contain 'high', 'low', and 'close' columns.")
    df['is_perfect_order'] = (df['sma10'] > df['sma20']) & (df['sma20'] > df['sma50']) & (df['sma50'] > df['sma100']) & (df['sma100'] > df['sma200'])
    df['is_prev_perfect_order'] = df['is_perfect_order'].shift(1)
    # 当前一个不是完美订单，而今天是完美订单，则买入
    df['buy_sig'] = np.where((df['is_perfect_order'] == True) & (df['is_prev_perfect_order'] == False), 0, 1)
    df['sell_sig'] = np.where((df['is_perfect_order'] == False) & (df['is_prev_perfect_order'] == True), 0, 1)

    # 
    return df

df = perfect_order_strategy(df)
df.dropna(inplace=True)
df.dtypes

datetime                  object
symbol                    object
open                     float64
high                     float64
low                      float64
close                    float64
volume                   float64
adx                      float64
sma10                    float64
sma20                    float64
sma50                    float64
sma100                   float64
sma200                   float64
is_perfect_order            bool
is_prev_perfect_order     object
buy_sig                    int64
sell_sig                   int64
dtype: object

In [29]:
new_df = df[['datetime', 'open', 'high', 'low', 'close', 'volume', 'buy_sig', 'sell_sig']].copy()
new_df['stop_loss'] = 0.0
# Rename 'datetime' column to ensure it's in the desired format
new_df['datetime'] = pd.to_datetime(new_df['datetime'])

# Set the data types for the new DataFrame
new_df = new_df.astype({
    'datetime': 'datetime64[ns]',
    'open': 'float64',
    'high': 'float64',
    'low': 'float64',
    'close': 'float64',
    'volume': 'int64',   # Change to int64 as specified
    'buy_sig': 'int64',
    'sell_sig': 'int64',
    'stop_loss': 'float64'  # Assuming you want this column as well
})

new_df.dtypes

datetime     datetime64[ns]
open                float64
high                float64
low                 float64
close               float64
volume                int64
buy_sig               int64
sell_sig              int64
stop_loss           float64
dtype: object